In [19]:
import os
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from textblob import TextBlob

In [20]:
SCOPES =["https://www.googleapis.com/auth/spreadsheets"]

spreadsheet_id = "1pq4y1R4ms6oQdkQ9Nb_FJLPWLj9nDC135RKhTdBNdTY"

In [38]:
def main():
    credentials = None
    if os.path.exists("token.json"):
        credentials = Credentials.from_authorized_user_file("token.json", SCOPES)
    if not credentials or not credentials.valid:
        if credentials and credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file("/Users/abhi/Desktop/pyauto/credentials.json", SCOPES)
            credentials = flow.run_local_server(port=0)
        with open("token.json", "w") as token:
            token.write(credentials.to_json())

    try:
        service = build("sheets", "v4", credentials=credentials)
        sheets = service.spreadsheets()

        from textblob import TextBlob

        def analyze_sentiment(feedback):
            blob = TextBlob(feedback)
            polarity = blob.sentiment.polarity
            if polarity > 0:
                return "Positive"
            elif polarity < 0:
                return "Negative"
            else:
                return "Neutral"
        sheet_metadata = sheets.get(spreadsheetId=spreadsheet_id, ranges=[]).execute()
        sheet_list = sheet_metadata.get("sheets", [])

        for sheet in sheet_list:
            sheet_title = sheet.get("properties", {}).get("title", "Sheet")
            if sheet_title == "Form Responses 1":
                sheet_id = sheet.get("properties", {}).get("sheetId")
                result = sheets.values().get(spreadsheetId=spreadsheet_id, range=f"{sheet_title}!A1:P").execute()
                values = result.get("values", [])

                sentiment_values = []
                for row in values:
                    if len(row) >= 16:  # Assuming the feedback column is the 16th column
                        feedback = row[15]
                        sentiment = analyze_sentiment(feedback)
                        sentiment_values.append([sentiment])
                update_body = {
                    "valueInputOption": "USER_ENTERED",
                    "data": [{"range": f"{sheet_title}!Q1:Q{len(sentiment_values)}", "values": sentiment_values}]
                }
                response = sheets.values().batchUpdate(spreadsheetId=spreadsheet_id, body=update_body).execute()
                print(f"Batch update response: {response}")  #Added print statement for better knowledge of updated batch response

    except HttpError as error:
        print(error)

if __name__ == "__main__":
    main()

Batch update response: {'spreadsheetId': '1pq4y1R4ms6oQdkQ9Nb_FJLPWLj9nDC135RKhTdBNdTY', 'totalUpdatedRows': 53, 'totalUpdatedColumns': 1, 'totalUpdatedCells': 53, 'totalUpdatedSheets': 1, 'responses': [{'spreadsheetId': '1pq4y1R4ms6oQdkQ9Nb_FJLPWLj9nDC135RKhTdBNdTY', 'updatedRange': "'Form Responses 1'!Q1:Q53", 'updatedRows': 53, 'updatedColumns': 1, 'updatedCells': 53}]}
